In [ ]:
!pip install transformers
!pip install datasets
!pip install adapter-transformers
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
%cd /content/drive/MyDrive/data/
# ! rm gpt2_adapter_casual
# ! mkdir gpt2_adapter_casual
! ls # verify that you are in the right directory

/content/drive/MyDrive/data
finalWords.json		  gpt2_predict_model
finetune_eval.txt	  gpt2_predict_model_30
finetune_train.txt	  gpt2_predict_model_6
finetune_val.txt	  predict_embeddings_model_weights
gpt2_adapter_casual	  predict_embeddings_model_weights_30
gpt2_adapter_mlm_model	  predict_embeddings_model_weights_6
gpt2_adapter_mlm_model_3  quiz_questions_10.txt
gpt2_casual		  quiz_questions_25.txt
gpt2_casual_2		  quiz_questions_2.txt
gpt2_finetune_random	  quiz_questions_50.txt
gpt2_mlm_model		  quiz_questions_5.txt
gpt2_mlm_model_2	  quiz_questions.txt
gpt2_mlm_model_3	  words.txt


In [ ]:
from transformers import AutoTokenizer, GPT2DoubleHeadsModel
import torch
import json


# Load the JSON file
with open('finalWords.json') as f:
    words = json.load(f)

model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = GPT2DoubleHeadsModel.from_pretrained("gpt2")
model = model.to(device)

original_vocab_size = len(tokenizer)
new_word_list = [i.strip().lower() for i in open("words.txt").readlines()]
num_new_words = len(new_word_list)
num_added = tokenizer.add_tokens(new_word_list)
model.resize_token_embeddings(len(tokenizer))

#Compute the distribution from which we’ll sample:
params = model.state_dict()
embeddings = params['transformer.wte.weight']
pre_expansion_embeddings = embeddings[:-num_new_words,:]
mu = torch.mean(pre_expansion_embeddings, dim=0)
n = pre_expansion_embeddings.size()[0]
sigma = ((pre_expansion_embeddings - mu).T @ (pre_expansion_embeddings - mu)) / n
dist = torch.distributions.multivariate_normal.MultivariateNormal(
        mu, covariance_matrix=1e-5*sigma)

#load in our new embeddings into the model:
new_embeddings = torch.stack(tuple((dist.sample() for _ in range(num_new_words))), dim=0)
embeddings[-num_new_words:,:] = new_embeddings
params['transformer.wte.weight'][-num_new_words:,:] = new_embeddings
model.load_state_dict(params)


Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-g

<All keys matched successfully>

In [ ]:
model.add_adapter("adapter", config="pfeiffer+inv")
model.train_adapter("adapter")

Adding adapter 'adapter'.


In [ ]:
from transformers import GPT2Tokenizer
from datasets import load_dataset
text = load_dataset("text",data_files={"train": "finetune_train.txt", "test": "finetune_val.txt"})
# print(text['train'])
# text = text['train'].train_test_split(test_size=0.2)
# print(text["train"][0:10])
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# tokenizer.pad_token = tokenizer.eos_token
def preprocess_function(examples):
    return tokenizer([x for x in examples["text"]], truncation=True)
tokenized_txt = text.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=text["train"].column_names
)
# print(tokenized_txt["train "][0:100])
block_size = 128

# print(tokenizer("My wife had a PMS Strike last night. PMS strikes are ruining my marriage!"))
def group_texts(examples):
    # print(examples)
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result
lm_dataset = tokenized_txt.map(group_texts, batched=True, num_proc=4)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
tokenizer.mask_token = u"\u2047"

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
from transformers import GPT2LMHeadModel
from transformers import TrainingArguments, AdapterTrainer

# model = GPT2LMHeadModel.from_pretrained('gpt2')
training_args = TrainingArguments(
    output_dir="gpt2_adapter_casual",
    evaluation_strategy="epoch",
    save_steps = 100000000,
    learning_rate=3e-4,
    num_train_epochs=12,
    weight_decay=0.1,
    logging_steps=200
)

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()
trainer.save_model("gpt2_adapter_casual")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 24682
  Num Epochs = 12
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 37032
  Number of trainable parameters = 1191361
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,5.345900,No log
2,5.256200,No log
3,5.224900,No log
4,5.171800,No log
5,5.164900,No log
6,5.117100,No log
7,5.113800,No log
8,5.123100,No log
9,5.106600,No log
10,5.098200,No log


***** Running Evaluation *****
  Num examples = 2873
  Batch size = 8
***** Running Evaluation *****
  Num examples = 2873
  Batch size = 8
***** Running Evaluation *****
  Num examples = 2873
  Batch size = 8
***** Running Evaluation *****
  Num examples = 2873
  Batch size = 8
***** Running Evaluation *****
  Num examples = 2873
  Batch size = 8
***** Running Evaluation *****
  Num examples = 2873
  Batch size = 8
***** Running Evaluation *****
  Num examples = 2873
  Batch size = 8
***** Running Evaluation *****
  Num examples = 2873
  Batch size = 8
***** Running Evaluation *****
  Num examples = 2873
  Batch size = 8
***** Running Evaluation *****
  Num examples = 2873
  Batch size = 8
***** Running Evaluation *****
  Num examples = 2873
  Batch size = 8
***** Running Evaluation *****
  Num examples = 2873
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to gpt2_adapter_casual
Configuration saved in gpt2

In [ ]:
#https://github.com/ayoolaolafenwa/TrainNLP

from accelerate import Accelerator
from torch.optim import AdamW
from transformers import get_scheduler
from torch.utils.data import DataLoader
batch_size = 16
train_dataloader = DataLoader(lm_dataset["train"], shuffle=True, batch_size=batch_size, collate_fn=data_collator,)

# load the test dataset for evaluation
eval_dataloader = DataLoader(lm_dataset["test"], batch_size=batch_size, collate_fn=data_collator)
eval_dataset = lm_dataset["test"]
# initialize pretrained bert model
# model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

# set the optimizer
optimizer = AdamW(model.parameters(), lr=3e-4)

# initialize accelerator for training
accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

# set the number of epochs which is set to 30
num_train_epochs = 10
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

# define the learning rate scheduler for training
lr_scheduler = get_scheduler("linear",optimizer=optimizer,num_warmup_steps=0,num_training_steps=num_training_steps)


In [ ]:

# model = GPT2DoubleHeadsModel.from_pretrained("gpt2")
optimizer = AdamW(model.parameters(), lr=3e-4)

checkpoint = torch.load("gpt2_adapter_casual/6")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

In [ ]:
import torch
import math
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

# directory to save the models
output_dir = "gpt2_adapter_casual/"
val_loss = 1000000
for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        losses.append(accelerator.gather(loss.repeat(batch_size)))

    losses = torch.cat(losses)
    losses = losses[: len(eval_dataset)]

    # perplexity metric used for mask language model training
    try:
        perplexity = math.exp(torch.mean(losses))
    except OverflowError:
        perplexity = float("inf")
    print(f">>> Epoch {epoch}: Perplexity: {perplexity}")

    # Save model
    accelerator.wait_for_everyone()
    torch.save({'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': perplexity}, 
  output_dir + str(epoch))
    if val_loss < perplexity:
      # model.save_pretrained(output_dir)
      # tokenizer.save_pretrained(output_dir+ "/tokenizer.json")
      print("done training")
      break
    val_loss = perplexity


  0%|          | 0/15430 [00:00<?, ?it/s]

>>> Epoch 0: Perplexity: 170.20959705920922
>>> Epoch 1: Perplexity: 167.15081427469474
>>> Epoch 2: Perplexity: 165.91600504022688
>>> Epoch 3: Perplexity: 164.31035525451045
>>> Epoch 4: Perplexity: 163.57466210492646
>>> Epoch 5: Perplexity: 162.83636175701213
>>> Epoch 6: Perplexity: 162.32106301065207


In [ ]:
model.save_adapter("gpt2_adapter_casual", "adapter", with_head=True)
tokenizer.save_pretrained("gpt2_adapter_casual/tokenizer.json")

Configuration saved in gpt2_adapter_casual/adapter_config.json
Module weights saved in gpt2_adapter_casual/pytorch_adapter.bin
Configuration saved in gpt2_adapter_casual/head_config.json
Module weights saved in gpt2_adapter_casual/pytorch_model_head.bin
tokenizer config file saved in gpt2_adapter_casual/tokenizer.json/tokenizer_config.json
Special tokens file saved in gpt2_adapter_casual/tokenizer.json/special_tokens_map.json


('gpt2_adapter_casual/tokenizer.json/tokenizer_config.json',
 'gpt2_adapter_casual/tokenizer.json/special_tokens_map.json',
 'gpt2_adapter_casual/tokenizer.json/vocab.json',
 'gpt2_adapter_casual/tokenizer.json/merges.txt',
 'gpt2_adapter_casual/tokenizer.json/added_tokens.json',
 'gpt2_adapter_casual/tokenizer.json/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, GPT2DoubleHeadsModel

model = GPT2DoubleHeadsModel.from_pretrained('gpt2')
tokenizer = AutoTokenizer.from_pretrained('gpt2')

original_vocab_size = len(tokenizer)
new_word_list = [i.strip().lower() for i in open("words.txt").readlines()]
num_new_words = len(new_word_list)
num_added = tokenizer.add_tokens(new_word_list)
model.resize_token_embeddings(len(tokenizer))

#Compute the distribution from which we’ll sample:
params = model.state_dict()
embeddings = params['transformer.wte.weight']
pre_expansion_embeddings = embeddings[:-num_new_words,:]
mu = torch.mean(pre_expansion_embeddings, dim=0)
n = pre_expansion_embeddings.size()[0]
sigma = ((pre_expansion_embeddings - mu).T @ (pre_expansion_embeddings - mu)) / n
dist = torch.distributions.multivariate_normal.MultivariateNormal(
        mu, covariance_matrix=1e-5*sigma)

#load in our new embeddings into the model:
new_embeddings = torch.stack(tuple((dist.sample() for _ in range(num_new_words))), dim=0)
embeddings[-num_new_words:,:] = new_embeddings
params['transformer.wte.weight'][-num_new_words:,:] = new_embeddings
model.load_state_dict(params)

adapter_name = model.load_adapter("gpt2_adapter_mlm_model/adapter")
model.set_active_adapters(adapter_name)
model = model.to(device)


Some weights of GPT2DoubleHeadsModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['multiple_choice_head.summary.weight', 'multiple_choice_head.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import json

num_added_tokens = tokenizer.add_special_tokens({"cls_token": "[CLS]"})
# Update the model embeddings with the new vocabulary size
# print("tokenizer length", len(tokenizer))
embedding_layer = model.resize_token_embeddings(len(tokenizer))

num_choices_arr = [5]
model_ans = []
for num_choices in num_choices_arr:

  with open(f"quiz_questions_{num_choices}.txt") as f:
    quiz_questions = json.load(f)
  print("NUM_CHOICES:", num_choices)

 

  questions = quiz_questions
  # answers = [0 for i in range(0, questions)]

  counts = [0.0 for i in range(0, num_choices)]
  total = 0.0
  # print("question", len(questions))
  model.eval()
  with torch.no_grad():        
    for i, choices in enumerate(questions):
        #print(choices)
        # print(i)
        if i % 1000 == 0 and i != 0:
          print(f"On iteration {i}, accuracy is {counts[0]/total} equal would be {1/num_choices} count is {total}", [i/total for i in counts])
        encoded_choices = [tokenizer.encode(s) for s in choices]
        cls_token_location = [tokens.index(tokenizer.cls_token_id) for tokens in encoded_choices]
        #check if some choices are longer than the others
        max_len = max([len(tokens) for tokens in encoded_choices])
        check = True
        for tokens in encoded_choices:
            if len(tokens) < max_len:
                check = False
                break
        if check:
            input_ids = torch.tensor(encoded_choices).unsqueeze(0)  # Batch size: 1, number of choices: 2
            if len(input_ids) >= 1024:
              continue
            mc_token_ids = torch.tensor([cls_token_location])  # Batch size: 1
            input_ids = input_ids.to(device)
            mc_token_ids = mc_token_ids.to(device)
            outputs = model(input_ids, mc_token_ids=mc_token_ids)
            lm_logits = outputs.logits
            mc_logits = outputs.mc_logits
            counts[mc_logits.argmax().item()] += 1
            total += 1
            model_ans.append(mc_logits.argmax().item())
        else:
          model_ans.append(-1)
        torch.cuda.empty_cache() 
  print(f"FINAL for num_choices {num_choices}: accuracy is {counts[0]/total} equal would be {1/num_choices} count is {total}", [i/total for i in counts])

  # print(counts)
  # print(total)

Assigning [CLS] to the cls_token key of the tokenizer


NUM_CHOICES: 5
On iteration 1000, accuracy is 0.18756268806419257 equal would be 0.2 count is 997.0 [0.18756268806419257, 0.20060180541624875, 0.18756268806419257, 0.21865596790371114, 0.20561685055165496]
On iteration 2000, accuracy is 0.19808467741935484 equal would be 0.2 count is 1984.0 [0.19808467741935484, 0.19556451612903225, 0.19102822580645162, 0.21421370967741934, 0.20110887096774194]
On iteration 3000, accuracy is 0.19912693082605776 equal would be 0.2 count is 2978.0 [0.19912693082605776, 0.19509738079247818, 0.19778374748153124, 0.20416386836803224, 0.2038280725319006]
On iteration 4000, accuracy is 0.2047858942065491 equal would be 0.2 count is 3970.0 [0.2047858942065491, 0.19672544080604534, 0.19269521410579346, 0.2035264483627204, 0.20226700251889168]
On iteration 5000, accuracy is 0.20757758968158002 equal would be 0.2 count is 4962.0 [0.20757758968158002, 0.19447803305118905, 0.198105602579605, 0.198105602579605, 0.20173317210802097]
On iteration 6000, accuracy is 0.2

In [ ]:
f = open("compare_5/adapter.txt", "w+")
for i in model_ans:
  f.write(str(i)+"\n")
f.close()
print(len(model_ans))

12211


In [ ]:
!ls compare_5

adapter.txt


In [ ]:
for i in range(0, 3):
  print(quiz_questions[i])
  print(quiz_answers[i])

['that kid is such a cuffy [CLS]', 'that kid is such a cuffy [CLS]', 'that kid is such a cuffy [CLS]', 'that kid is such a cuffy [CLS]', 'that kid is such a cuffy [CLS]']
0
["Let's go spooch one down brah [CLS]", "Let's go nussy one down brah [CLS]", "Let's go Noiners one down brah [CLS]", "Let's go dizzle one down brah [CLS]", "Let's go civic one down brah [CLS]"]
0
['Dude, did that (American Muscle/Opel/just about any performance vehicle) just completely obliterate that WRX? [CLS]', 'Dude, did that (American Muscle/Opel/just about any performance vehicle) just completely obliterate that WRX? [CLS]', 'Dude, did that (American Muscle/Opel/just about any performance vehicle) just completely obliterate that WRX? [CLS]', 'Dude, did that (American Muscle/Opel/just about any performance vehicle) just completely obliterate that WRX? [CLS]', 'Dude, did that (American Muscle/Opel/just about any performance vehicle) just completely obliterate that WRX? [CLS]']
0


In [ ]:
# Add a [CLS] to the vocabulary (we should train it also!)
num_added_tokens = tokenizer.add_special_tokens({"cls_token": "[CLS]"})
# Update the model embeddings with the new vocabulary size
print(len(tokenizer))
embedding_layer = model.resize_token_embeddings(len(tokenizer))

questions = quiz_questions
answers = quiz_answers

count0 = 0.0
count1 = 0.0
count2 = 0.0
count3 = 0.0
count4 = 0.0
total = 0.0
print("question", len(questions))
model.eval()
with torch.no_grad():        
  for i, choices in enumerate(questions):
      #print(choices)
      # print(i)
      if i % 100 == 0:
        print(i)
      encoded_choices = [tokenizer.encode(s) for s in choices]
      cls_token_location = [tokens.index(tokenizer.cls_token_id) for tokens in encoded_choices]
      #check if some choices are longer than the others
      max_len = max([len(tokens) for tokens in encoded_choices])
      check = True
      for tokens in encoded_choices:
          if len(tokens) < max_len:
              check = False
      if check:
          input_ids = torch.tensor(encoded_choices).unsqueeze(0)  # Batch size: 1, number of choices: 2
          if len(input_ids) >= 1024:
            continue
          mc_token_ids = torch.tensor([cls_token_location])  # Batch size: 1
          input_ids = input_ids.to(device)
          mc_token_ids = mc_token_ids.to(device)
          outputs = model(input_ids, mc_token_ids=mc_token_ids)
          lm_logits = outputs.logits
          mc_logits = outputs.mc_logits
          if mc_logits.argmax().item() == 0:
              count0 += 1
          elif mc_logits.argmax().item() == 1:
              count1 += 1
          elif mc_logits.argmax().item() == 2:
              count2 += 1
          elif mc_logits.argmax().item() == 3:
              count3 += 1
          elif mc_logits.argmax().item() == 4:
              count4 += 1
          
          total += 1

print(len(questions))
print(count0)
print(count1)
print(count2)
print(count3)
print(count4)
print(total)


Assigning [CLS] to the cls_token key of the tokenizer


79247
question 4433
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4433
815.0
714.0
688.0
686.0
727.0
3630.0
